### Alterative aproaches for some functions
Initial tests showed they are slower, but that could be verifed.

In [ ]:

def get_all_solutions2(k):
    '''
    Generates all encoded polygons of size k, including all rotations.
    Args:
        k (int): The size of polygon to be made.
    Returns:
        list[tuple[int]]: A list of tuples, where each tuple represents the encoding of suitable polygon.
    '''
    solutions = get_unique_solutions(k)
    result = []
    for solution in solutions:
        solution_permutations = [(solution[0],) + perm for perm in set(permutations(solution[1:]))]
        result.extend(solution_permutations)
    return result

def expand_to_left2(triangle, k):
    '''
    Expands given triangle of size n, in matrix-like representation, to a triangle of size n+k 
    by adding 2*k cells at the beginning of each row, extending the base length to n+k, and 
    adding a new triangle of size k at the top to maintain the triangle shape.
    Args:
        triangle (list[list[int]]): A matrix-like representation of the initial triangle to expand.
        k (int): The number of units to add to the left side of each row.
    Returns:
        list[list[int]]: A matrix-like representation of expanded triangle.
    '''   
    for _ in range(k):
        for i in range(len(triangle)):
            triangle[i] = [0, 0] + triangle[i]
        triangle.insert(0, [0])
    return triangle

def expand_to_right(triangle, k):
    '''
    Expands given triangle of size n, in matrix-like representation, to a triangle of size n+k 
    by adding 2*k cells to the end of each row, extending the base length to n+k, and 
    adding a new triangle of size k at the top to maintain the triangle shape.
    Args:
        triangle (list[list[int]]): A matrix-like representation of the initial triangle to expand.
        k (int): The number of units to add to the right side of each row.
    Returns:
        list[list[int]]: A matrix-like representation of expanded triangle.
    '''   
    for _ in range(k):
        for i in range(len(triangle)):
            triangle[i] = triangle[i] + [0, 0]    
        triangle.insert(0, [0])
    return triangle

### Move performation related functions
Functions below operate on matrix-like representations, which turns out to be suboptimal approach, but they are saved just in case 

In [ ]:
def is_placement_valid(board, triangle):
    '''
    Checks if a triangle can be placed on the board.
    Performs an element-wise logical AND on the NumPy array representations of the board and triangle.
    The result gives the points of intersection between the triangle and the board. 
    If the number of intersection points is zero, the placement is valid.
    
    Args:
        triangle (list[list[int]]): A matrix-like representation of a triangle.
        board (list[list[int]]): A matrix-like representation of a triangular board.
    
    Returns:
        bool: True if triangle can be placed on board, False otherwise.
    '''
    board = convert_triangle_to_numpy_array(board, 0)
    triangle = convert_triangle_to_numpy_array(triangle, 0)
    assert board.shape == triangle.shape, f'Shapes do not match. Board shape: {board.shape}, triangle shape: {triangle.shape}'

    return not np.logical_and(
        board, triangle
    ).sum() # True only if the sum is equal to 0.


def get_possible_placements(board, k):
    '''
    Returns all possible placements of k-element polygons on the given board.

    Args:
        board (list[list[int]]): A matrix-like representation of a triangular board.
        k (int): The number of elements in the polygon to be placed.
    
    Returns:
        list[list[list[int]]]: A list of matrix-like representations for all possible placements of k-element polygons on the board.
    '''

    board_matrix = convert_triangle_to_numpy_array(board)
    n = board_matrix.shape[0] 
    empty_cells_num = n**2 - board_matrix.sum()
    result = []
    if k > empty_cells_num:  
        return result
    for polygon in get_all_solutions(k):
        for expanded_polygon in expand_polygon(*polygon, n):
            if is_placement_valid(board, expanded_polygon):
                result.append(expanded_polygon)
    return result


def get_possible_moves(board, turn):
    '''
    Returns all possible moves on the given board for the specified turn.

    Args:
        board (list[list[int]]): A matrix-like representation of a triangular board.
        turn (int): The current turn.
    
    Returns:
        list[list[list[int]]]: A list of matrix-like representations of all possible moves.
    '''

    return get_possible_placements(board, turn) + get_possible_placements(board, turn+1)
